In [1]:
!pip install python-dotenv
!pip install requests
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # Carrega o .env do diretório atual


True

In [2]:
import zipfile
import os

# Caminhos
zip_file_path = "/content/202401_NFs.zip"  # Arquivo ZIP
extraction_path = "/content/dados_extraidos/"  # Pasta de destino

# Criar pasta se não existir
os.makedirs(extraction_path, exist_ok=True)

# Extrair
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)
    print(f"Arquivos extraídos em: {extraction_path}")

# Verificar conteúdo
!ls -lh {extraction_path}

Arquivos extraídos em: /content/dados_extraidos/
total 336K
-rw-r--r-- 1 root root  43K Jun 18 01:13 202401_NFs_Cabecalho.csv
-rw-r--r-- 1 root root 292K Jun 18 01:13 202401_NFs_Itens.csv


In [3]:
!pip install python-dotenv
from dotenv import load_dotenv
import os

load_dotenv()  # Carrega o arquivo .env
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")


In [8]:
import sys
!{sys.executable} -m pip install langchain-experimental
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import glob
import pandas as pd

diretorio = glob.glob("dados_extraidos/*.csv")
dfs_csv = []

for arquivo in diretorio:
    df = pd.read_csv(arquivo)
    df['Arquivo_Origem'] = arquivo.split('/')[-1]  # Adiciona o nome do arquivo como coluna
    dfs_csv.append(df)

df_final = pd.concat(dfs_csv, ignore_index=True)
print(df_final.head())


template = """
Você é um analista contabil .
Sempre responda com base em dfs_csv , só reponda sobre NF e item com base em dfs_csv .
Não tente inventar uma resposta

Pergunta: {question}
"""
prompt = PromptTemplate.from_template(template)


# 1. Configuração segura do agente
agent_executor = create_pandas_dataframe_agent(
    ChatOpenAI(
        openai_api_key=deepseek_api_key,
        openai_api_base="https://api.deepseek.com/v1",
        model_name="deepseek-chat",
        temperature=0.3,
        max_tokens=2000
    ),
    dfs_csv,  # Seu DataFrame
    prompt=prompt,
    verbose=True,
    #agent_type="openai-tools",
    max_iterations=10,  # Limita o número de operações
    allow_dangerous_code=True,  # Permite execução controlada
    handle_parsing_errors=True,  # Melhor tratamento de erros
    early_stopping_method="generate"  # Interrompe se demorar muito
)


# Exemplo de uso
resposta = agent_executor.invoke({"input": "Quantos registros existem?"})
print(resposta["output"])

pergunta = "Quantas notas fiscais existem no total?"
resposta = agent_executor.invoke({"input": pergunta})

print(f"Pergunta: {pergunta}")
print(f"Resposta: {resposta.get('output')}")



                                CHAVE DE ACESSO  \
0  41240106267630001509550010035101291224888487   
1  50240129843878000170550010000025251000181553   
2  50240129843878000170550010000025251000181553   
3  50240129843878000170550010000025251000181553   
4  50240112977901000117550010000051831659469117   

                                              MODELO  SÉRIE   NÚMERO  \
0  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1  3510129   
1  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
2  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
3  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
4  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     5183   

                   NATUREZA DA OPERAÇÃO         DATA EMISSÃO  \
0  Outras Entradas - Dev Remessa Escola  2024-01-18 07:10:39   
1    VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42   
2    VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42   
3   

/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'prompt': PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\nVocê é um analista contabil .\nSempre responda com base em dfs_csv , só reponda sobre NF e item com base em dfs_csv .\nNão tente inventar uma resposta\n\nPergunta: {question}\n'), 'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


Thought: Para responder à pergunta sobre quantos registros existem em cada dataframe, posso usar a função `len()` do pandas para contar o número de linhas em cada um.

Action: python_repl_ast
Action Input: len(df1)565Agora vou verificar o número de registros no segundo dataframe.

Action: python_repl_ast
Action Input: len(df2)100Agora tenho a informação completa sobre o número de registros em cada dataframe.

Final Answer: O dataframe df1 possui 565 registros e o dataframe df2 possui 100 registros.

> Finished chain.
O dataframe df1 possui 565 registros e o dataframe df2 possui 100 registros.


> Entering new AgentExecutor chain...
Thought: Para contar o número total de notas fiscais, precisamos verificar quantas linhas únicas existem em cada dataframe, considerando que cada linha representa uma nota fiscal ou um item de nota fiscal. Como o primeiro dataframe parece conter itens de notas fiscais e o segundo contém cabeçalhos de notas fiscais, o mais preciso seria contar as notas fiscai

In [9]:
pergunta = "o nome do item com maior valor e me apresente esse valor ?"
resposta = agent_executor.invoke({"input": pergunta})



> Entering new AgentExecutor chain...
Para responder à pergunta, precisamos encontrar o item com o maior valor na coluna "VALOR TOTAL" do dataframe de itens (df1) e retornar sua descrição e valor. Vou executar os passos necessários:

Thought: Preciso encontrar a linha com o maior valor na coluna "VALOR TOTAL" no dataframe df1 e retornar a descrição do produto e o valor.
Action: python_repl_ast
Action Input: 
```python
max_value_row = df1.loc[df1['VALOR TOTAL'].idxmax()]
max_item = max_value_row['DESCRIÇÃO DO PRODUTO/SERVIÇO']
max_value = max_value_row['VALOR TOTAL']
print(f"Item: {max_item}\nValor: {max_value}")
```
Item: HGC 4 Branco/Branco-300-CP
Valor: 985050.0
I now know the final answer
Final Answer: O item com maior valor é "HGC 4 Branco/Branco-300-CP" com o valor de R$985.050,00.

> Finished chain.
